<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ резюме из HeadHunter
   

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Исследование структуры данных

1. Прочитайте данные с помощью библиотеки Pandas. Совет: перед чтением обратите внимание на разделитель внутри файла. 

In [ ]:
hh_data = pd.read_csv('dst-3.0_16_1_hh_database.csv', sep=';')
razmer = (hh_data.shape[0], hh_data.shape[1])
display(razmer) 

2. Выведите несколько первых (последних) строк таблицы, чтобы убедиться в том, что ваши данные не повреждены. Ознакомьтесь с признаками и их структурой.

In [ ]:
hh_data.head()

3. Выведите основную информацию о числе непустых значений в столбцах и их типах в таблице.

In [ ]:
hh_data.info()

4. Обратите внимание на информацию о числе непустых значений. 

In [ ]:
cols_null_percent = hh_data.isnull().mean() * 100
cols_with_null = cols_null_percent[cols_null_percent>0].sort_values(ascending=False)
display(cols_with_null)

5. Выведите основную статистическую информацию о столбцах.


In [ ]:
hh_data.describe()

# Преобразование данных

1. Начнем с простого - с признака **"Образование и ВУЗ"**. Его текущий формат это: **<Уровень образования год выпуска ВУЗ специальность...>**. Например:
* Высшее образование 2016 Московский авиационный институт (национальный исследовательский университет)...
* Неоконченное высшее образование 2000  Балтийская государственная академия рыбопромыслового флота…
Нас будет интересовать только уровень образования.

Создайте с помощью функции-преобразования новый признак **"Образование"**, который должен иметь 4 категории: "высшее", "неоконченное высшее", "среднее специальное" и "среднее".

Выполните преобразование, ответьте на контрольные вопросы и удалите признак "Образование и ВУЗ".

Совет: обратите внимание на структуру текста в столбце **"Образование и ВУЗ"**. Гарантируется, что текущий уровень образования соискателя всегда находится в первых 2ух слов и начинается с заглавной буквы. Воспользуйтесь этим.

*Совет: проверяйте полученные категории, например, с помощью метода unique()*


In [ ]:
hh_data_w = hh_data.copy()
hh_data_w.head()

In [ ]:
def obraz(shul): 
    shul_spis = shul.split(' ')
    n = len(shul_spis)
    slovo = ''
    if n > 3:
        n = 3
    for i in range(n):
        slovo = slovo + shul_spis[i] + ' '
    if 'Неоконченное высшее' in slovo:
        return 'неоконченное высшее'
    elif 'Высшее' in slovo:
        return 'высшее'
    elif 'Среднее специальное' in slovo:
        return 'среднее специальное'
    elif 'Среднее' in slovo:
        return 'среднее'
    else:
        return ' '
    
hh_data_w['Образование'] = hh_data_w['Образование и ВУЗ'].apply(obraz)       


In [ ]:
display(hh_data_w[hh_data_w['Образование'] == 'среднее'].shape[0])

In [ ]:
hh_data_w = hh_data_w.drop('Образование и ВУЗ', axis=1)

In [ ]:
hh_data_w.head()

2. Теперь нас интересует столбец **"Пол, возраст"**. Сейчас он представлен в формате **<Пол , возраст , дата рождения >**. Например:
* Мужчина , 39 лет , родился 27 ноября 1979 
* Женщина , 21 год , родилась 13 января 2000
Как вы понимаете, нам необходимо выделить каждый параметр в отдельный столбец.

Создайте два новых признака **"Пол"** и **"Возраст"**. При этом важно учесть:
* Признак пола должен иметь 2 уникальных строковых значения: 'М' - мужчина, 'Ж' - женщина. 
* Признак возраста должен быть представлен целыми числами.

Выполните преобразование, ответьте на контрольные вопросы и удалите признак **"Пол, возраст"** из таблицы.

*Совет: обратите внимание на структуру текста в столбце, в части на то, как разделены параметры пола, возраста и даты рождения между собой - символом ' , '. 
Гарантируется, что структура одинакова для всех строк в таблице. Вы можете воспользоваться этим.*


In [ ]:
def f_pol(argument): 
    if "Мужчина" in argument:
        pol = 'M'
    elif "Женщина" in argument:
        pol = 'Ж'
    else:
        pol = ''    
    return pol    

def f_age(argument): 
    argument_spis = argument.split(',')
    age = argument_spis[1].replace(' ','')
    age = age[0:2]
    return int(age)    
       
hh_data_w['Пол'] = hh_data_w['Пол, возраст'].apply(f_pol)      
hh_data_w['Возраст'] = hh_data_w['Пол, возраст'].apply(f_age)     


In [ ]:
hh_data_w = hh_data_w.drop('Пол, возраст', axis=1)

In [ ]:
display(round((((hh_data_w[hh_data_w['Пол'] == 'Ж'].shape[0]) / (hh_data_w.shape[0])) * 100), 2))
display(round((hh_data_w['Возраст'].mean()), 1))

3. Следующим этапом преобразуем признак **"Опыт работы"**. Его текущий формат - это: **<Опыт работы: n лет m месяцев, периоды работы в различных компаниях…>**. 

Из столбца нам необходимо выделить общий опыт работы соискателя в месяцах, новый признак назовем "Опыт работы (месяц)"

Для начала обсудим условия решения задачи:
* Во-первых, в данном признаке есть пропуски. Условимся, что если мы встречаем пропуск, оставляем его как есть (функция-преобразование возвращает NaN)
* Во-вторых, в данном признаке есть скрытые пропуски. Для некоторых соискателей в столбце стоит значения "Не указано". Их тоже обозначим как NaN (функция-преобразование возвращает NaN)
* В-третьих, нас не интересует информация, которая описывается после указания опыта работы (периоды работы в различных компаниях)
* В-четвертых, у нас есть проблема: опыт работы может быть представлен только в годах или только месяцах. Например, можно встретить следующие варианты:
    * Опыт работы 3 года 2 месяца…
    * Опыт работы 4 года…
    * Опыт работы 11 месяцев…
    * Учитывайте эту особенность в вашем коде

Учитывайте эту особенность в вашем коде

В результате преобразования у вас должен получиться столбец, содержащий информацию о том, сколько месяцев проработал соискатель.
Выполните преобразование, ответьте на контрольные вопросы и удалите столбец **"Опыт работы"** из таблицы.


In [ ]:
def stag(argument):
    if argument is np.nan or argument == 'Не указано':
       return np.nan
    argument_spis = argument.split(' ')
    kol_mon = 0
    for i in range(7):
        if argument_spis[i] in year_spis:
            if argument_spis[i-1].isdigit():
                kol_mon += int(argument_spis[i-1]) * 12
        if argument_spis[i] in mon_spis:
            if argument_spis[i-1].isdigit():
                kol_mon += int(argument_spis[i-1])
    return kol_mon    
            
year_spis = ['год', 'года', 'лет']  
mon_spis = ['месяц', 'месяца', 'месяцев'] 
hh_data_w['Опыт работы (месяц)'] = hh_data_w['Опыт работы'].apply(stag)  
display(hh_data_w['Опыт работы (месяц)'].median())
hh_data_w = hh_data_w.drop('Опыт работы', axis=1)

4. Хорошо идем! Следующий на очереди признак "Город, переезд, командировки". Информация в нем представлена в следующем виде: **<Город , (метро) , готовность к переезду (города для переезда) , готовность к командировкам>**. В скобках указаны необязательные параметры строки. Например, можно встретить следующие варианты:

* Москва , не готов к переезду , готов к командировкам
* Москва , м. Беломорская , не готов к переезду, не готов к командировкам
* Воронеж , готов к переезду (Сочи, Москва, Санкт-Петербург) , готов к командировкам

Создадим отдельные признаки **"Город"**, **"Готовность к переезду"**, **"Готовность к командировкам"**. При этом важно учесть:

* Признак **"Город"** должен содержать только 4 категории: "Москва", "Санкт-Петербург" и "город-миллионник" (их список ниже), остальные обозначьте как "другие".

    Список городов-миллионников:
    
   <code>million_cities = ['Новосибирск', 'Екатеринбург','Нижний Новгород','Казань', 'Челябинск','Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск', 'Пермь', 'Воронеж','Волгоград']
    </code>
    Инфорация о метро, рядом с которым проживает соискатель нас не интересует.
* Признак **"Готовность к переезду"** должен иметь два возможных варианта: True или False. Обратите внимание, что возможны несколько вариантов описания готовности к переезду в признаке "Город, переезд, командировки". Например:
    * … , готов к переезду , …
    * … , не готова к переезду , …
    * … , готова к переезду (Москва, Санкт-Петербург, Ростов-на-Дону)
    * … , хочу переехать (США) , …
    
    Нас интересует только сам факт возможности или желания переезда.
* Признак **"Готовность к командировкам"** должен иметь два возможных варианта: True или False. Обратите внимание, что возможны несколько вариантов описания готовности к командировкам в признаке "Город, переезд, командировки". Например:
    * … , готов к командировкам , … 
    * … , готова к редким командировкам , …
    * … , не готов к командировкам , …
    
    Нас интересует только сам факт готовности к командировке.
    
    Еще один важный факт: при выгрузки данных у некоторых соискателей "потерялась" информация о готовности к командировкам. Давайте по умолчанию будем считать, что такие соискатели не готовы к командировкам.
    
Выполните преобразования и удалите столбец **"Город, переезд, командировки"** из таблицы.

*Совет: обратите внимание на то, что структура текста может меняться в зависимости от указания ближайшего метро. Учите это, если будете использовать порядок слов в своей программе.*


In [ ]:
def city(argument):
    argument_spis = argument.split(' ')
    n = len(argument_spis)
    for i in range(n):
        if argument_spis[i] in million_cities:
            return 'город-миллионник'
        elif argument_spis[i] == 'Москва':
            return 'Москва'
        elif argument_spis[i] == 'Санкт-Петербург':
            return 'Санкт-Петербург'
        else:
            return 'другие'
        
def pereezd(argument):
    bol = False
    if ('не готов к переезду' in argument) | ('не готова к переезду' in argument):
        bol = False
    elif ('готов к переезду' in argument) | ('готова к переезду' in argument) | ('хочу' in argument):   
        bol = True
    return bol

def kom(argument):
    bol = False
    if ('не готов к командировкам' in argument) | ('не готова к командировкам' in argument):
        bol = False
    elif ('готов к командировкам' in argument)  | ('готова к командировкам' in argument) | ('редким' in argument):   
        bol = True
    return bol
         
million_cities = ['Новосибирск', 'Екатеринбург', 'Нижний Новгород', 'Казань', 'Челябинск', 'Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск', 'Пермь', 'Воронеж', 'Волгоград' ]    
hh_data_w['Город'] = hh_data_w['Город, переезд, командировки'].apply(city) 
hh_data_w['Готовность к переезду'] = hh_data_w['Город, переезд, командировки'].apply(pereezd) 
hh_data_w['Готовность к командировкам'] = hh_data_w['Город, переезд, командировки'].apply(kom) 

display(round((((hh_data_w[hh_data_w['Город'] == 'Санкт-Петербург'].shape[0]) / (hh_data_w.shape[0])) * 100)))
mask1 = hh_data_w['Готовность к переезду'] == True
mask2 = hh_data_w['Готовность к командировкам'] == True
display(round(((hh_data_w[mask1 & mask2].shape[0]) / (hh_data_w.shape[0])) * 100))
hh_data_w = hh_data_w.drop('Город, переезд, командировки', axis=1)

In [ ]:
hh_data_w.head()

5. Рассмотрим поближе признаки **"Занятость"** и **"График"**. Сейчас признаки представляют собой набор категорий желаемой занятости (полная занятость, частичная занятость, проектная работа, волонтерство, стажировка) и желаемого графика работы (полный день, сменный график, гибкий график, удаленная работа, вахтовый метод).
На сайте hh.ru соискатель может указывать различные комбинации данных категорий, например:
* полная занятость, частичная занятость
* частичная занятость, проектная работа, волонтерство
* полный день, удаленная работа
* вахтовый метод, гибкий график, удаленная работа, полная занятость

Такой вариант признаков имеет множество различных комбинаций, а значит множество уникальных значений, что мешает анализу. Нужно это исправить!

Давайте создадим признаки-мигалки для каждой категории: если категория присутствует в списке желаемых соискателем, то в столбце на месте строки рассматриваемого соискателя ставится True, иначе - False.

Такой метод преобразования категориальных признаков называется One Hot Encoding и его схема представлена на рисунке ниже:
<img src=https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/ohe.jpg>
Выполните данное преобразование для признаков "Занятость" и "График", ответьте на контрольные вопросы, после чего удалите их из таблицы

In [ ]:
hh_data_w['полная занятость'] = hh_data_w['Занятость'].apply(lambda x: True if 'полн' in x else False)
hh_data_w['частичная занятость'] = hh_data_w['Занятость'].apply(lambda x: True if 'частич' in x else False)
hh_data_w['проектная работа'] = hh_data_w['Занятость'].apply(lambda x: True if 'проект' in x else False)
hh_data_w['стажировка'] = hh_data_w['Занятость'].apply(lambda x: True if 'стаж' in x else False)
hh_data_w['волонтерство'] = hh_data_w['Занятость'].apply(lambda x: True if 'волон' in x else False)

mask1 = hh_data_w['проектная работа'] == True
mask2 = hh_data_w['волонтерство'] == True
display(hh_data_w[mask1 & mask2].shape[0])

hh_data_w['полный день'] = hh_data_w['График'].apply(lambda x: True if 'полн' in x else False)
hh_data_w['сменный график'] = hh_data_w['График'].apply(lambda x: True if 'смен' in x else False)
hh_data_w['гибкий график'] = hh_data_w['График'].apply(lambda x: True if 'гибк' in x else False)
hh_data_w['удаленная работа'] = hh_data_w['График'].apply(lambda x: True if 'удал' in x else False)
hh_data_w['вахтовый метод'] = hh_data_w['График'].apply(lambda x: True if 'вахт' in x else False)

mask1 = hh_data_w['гибкий график'] == True
mask2 = hh_data_w['вахтовый метод'] == True
display(hh_data_w[mask1 & mask2].shape[0])

hh_data_w = hh_data_w.drop('Занятость', axis=1)
hh_data_w = hh_data_w.drop('График', axis=1)

6. (2 балла) Наконец, мы добрались до самого главного и самого важного - признака заработной платы **"ЗП"**. 
В чем наша беда? В том, что помимо желаемой заработной платы соискатель указывает валюту, в которой он бы хотел ее получать, например:
* 30000 руб.
* 50000 грн.
* 550 USD

Нам бы хотелось видеть заработную плату в единой валюте, например, в рублях. Возникает вопрос, а где взять курс валют по отношению к рублю?

На самом деле язык Python имеет в арсенале огромное количество возможностей получения данной информации, от обращения к API Центробанка, до использования специальных библиотек, например pycbrf. Однако, это не тема нашего проекта.

Поэтому мы пойдем в лоб: обратимся к специальным интернет-ресурсам для получения данных о курсе в виде текстовых файлов. Например, MDF.RU, данный ресурс позволяет удобно экспортировать данные о курсах различных валют и акций за указанные периоды в виде csv файлов. Мы уже сделали выгрузку курсов валют, которые встречаются в наших данных за период с 29.12.2017 по 05.12.2019. Скачать ее вы можете **на платформе**

Создайте новый DataFrame из полученного файла. В полученной таблице нас будут интересовать столбцы:
* "currency" - наименование валюты в ISO кодировке,
* "date" - дата, 
* "proportion" - пропорция, 
* "close" - цена закрытия (последний зафиксированный курс валюты на указанный день).


Перед вами таблица соответствия наименований иностранных валют в наших данных и их общепринятых сокращений, которые представлены в нашем файле с курсами валют. Пропорция - это число, за сколько единиц валюты указан курс в таблице с курсами. Например, для казахстанского тенге курс на 20.08.2019 составляет 17.197 руб. за 100 тенге, тогда итоговый курс равен - 17.197 / 100 = 0.17197 руб за 1 тенге.
Воспользуйтесь этой информацией в ваших преобразованиях.

<img src=https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/table.jpg>


Осталось только понять, откуда брать дату, по которой определяется курс? А вот же она - в признаке **"Обновление резюме"**, в нем содержится дата и время, когда соискатель выложил текущий вариант своего резюме. Нас интересует только дата, по ней бы и будем сопоставлять курсы валют.

Теперь у нас есть вся необходимая информация для того, чтобы создать признак "ЗП (руб)" - заработная плата в рублях.

После ответа на контрольные вопросы удалите исходный столбец заработной платы "ЗП" и все промежуточные столбцы, если вы их создавали.

Итак, давайте обсудим возможный алгоритм преобразования: 
1. Перевести признак "Обновление резюме" из таблицы с резюме в формат datetime и достать из него дату. В тот же формат привести признак "date" из таблицы с валютами.
2. Выделить из столбца "ЗП" сумму желаемой заработной платы и наименование валюты, в которой она исчисляется. Наименование валюты перевести в стандарт ISO согласно с таблицей выше.
3. Присоединить к таблице с резюме таблицу с курсами по столбцам с датой и названием валюты (подумайте, какой тип объединения надо выбрать, чтобы в таблице с резюме сохранились данные о заработной плате, изначально представленной в рублях). Значение close для рубля заполнить единицей 1 (курс рубля самого к себе)
4. Умножить сумму желаемой заработной платы на присоединенный курс валюты (close) и разделить на пропорцию (обратите внимание на пропуски после объединения в этих столбцах), результат занести в новый столбец "ЗП (руб)".


In [ ]:
kurs_data = pd.read_csv('ExchangeRates.csv', sep=',')
kurs_data.head()


In [ ]:
def data(argument):
    argument_spis = argument.split(' ')
    return argument_spis[0]
        
hh_data_w['Обновление резюме'] = hh_data_w['Обновление резюме'].apply(data) 
hh_data_w['Обновление резюме'] = pd.to_datetime(hh_data_w['Обновление резюме'])
kurs_data['date'] = pd.to_datetime(kurs_data['date'])


In [ ]:
def zp(argument):
    argument_spis = argument.split(' ')
    return int(argument_spis[0])
        
hh_data_w['ЗП сумма'] = hh_data_w['ЗП'].apply(zp) 

def valuta(argument):
    argument_spis = argument.split()
    x = argument_spis[1]
    currency = {
        'руб': 'RUB',
        'грн': 'UAH',
        'USD': 'USD',
        'белруб': 'BYN',
        'сум': 'UZS',
        'EUR': 'EUR',
        'KGS': 'KGS',
        'KZT': 'KZT',
        'UZS': 'UZS',
        'AZN': 'AZN'
    }
    x = x.replace(' ', '')
    x = x.replace('.', '')
    return currency[x]
    

hh_data_w['ЗП валюта'] = hh_data_w['ЗП'].apply(valuta)   

hh_data_w = hh_data_w.merge(
    kurs_data,                                                  
    left_on = ['ЗП валюта','Обновление резюме'],
    right_on = ['currency', 'date'],
    how = 'left')

hh_data_w['close'] = hh_data_w['close'].fillna(1)
hh_data_w['proportion'] = hh_data_w['proportion'].fillna(1)

hh_data_w['ЗП (руб)'] = hh_data_w['ЗП сумма'] * hh_data_w['close'] / hh_data_w['proportion']
display(hh_data_w['ЗП (руб)'].median())
hh_data_w = hh_data_w.drop(['ЗП валюта', 'ЗП сумма', 'currency', 'per', 'date', 'time', 'close', 'vol', 'proportion', 'ЗП'],axis=1)


In [ ]:
hh_data_w.head(1000)

In [ ]:
kurs_data.head()


In [ ]:
hh_data_w.head()


# Исследование зависимостей в данных

1. Постройте распределение признака **"Возраст"**. Опишите распределение, отвечая на следующие вопросы: чему равна мода распределения, каковы предельные значения признака, в каком примерном интервале находится возраст большинства соискателей? Есть ли аномалии для признака возраста, какие значения вы бы причислили к их числу?
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*

In [ ]:
data_graf = hh_data_w['Возраст']
fig = px.histogram(
    data_frame=data_graf, #DataFrame
    x='Возраст', #ось ординат
    height=500, #высота
    width=1000, #ширина
    marginal='box',
    title='Распределение возраста соискателей' #заголовок
)
fig.show()

display(data_graf.mode())

Модальное значение возраста - 30 лет.
Предельные значения признака от 10 до 77 лет.
Большинство соикателей от 21 до 40 лет.
Аномалия - 10 лет соискателю. 

2. Постройте распределение признака **"Опыт работы (месяц)"**. Опишите данное распределение, отвечая на следующие вопросы: чему равна мода распределения, каковы предельные значения признака, в каком примерном интервале находится опыт работы большинства соискателей? Есть ли аномалии для признака опыта работы, какие значения вы бы причислили к их числу?
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*

In [ ]:
data_graf = hh_data_w['Опыт работы (месяц)']
fig = px.histogram(
    data_frame=data_graf, #DataFrame
    x='Опыт работы (месяц)', #ось ординат
    height=500, #высота
    width=1000, #ширина
    marginal='box',
    title='Распределение опыта работы' #заголовок
)
fig.show()

display(data_graf.max())
display(data_graf.mode())

Мода распределения равна 81 месяцу.
Предельные признаки - от 1 до 1188 месяцев.
Опыт работы большинства - от 5 до 200 месяцев.
Аномалия - 1188 месяцев.

3. Постройте распределение признака **"ЗП (руб)"**. Опишите данное распределение, отвечая на следующие вопросы: каковы предельные значения признака, в каком примерном интервале находится заработная плата большинства соискателей? Есть ли аномалии для признака возраста? Обратите внимание на гигантские размеры желаемой заработной платы.
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*


In [ ]:
data_graf = hh_data_w['ЗП (руб)']
fig = px.histogram(
    data_frame=data_graf, #DataFrame
    x='ЗП (руб)', #ось ординат
    height=500, #высота
    width=1000, #ширина
    marginal='box',
    title='Распределение ЗП' #заголовок
)
fig.show()

display(data_graf.mode())
display((hh_data_w[hh_data_w['ЗП (руб)'] > 1000000].shape[0]))

Мода распределения равна 50000.
Предельные признаки - от 1 до 24304876 рублей.
ЗП большинства - от 37 до 95 тысяч.
Аномалия - 24304876 рублей.

4. Постройте диаграмму, которая показывает зависимость **медианной** желаемой заработной платы (**"ЗП (руб)"**) от уровня образования (**"Образование"**). Используйте для диаграммы данные о резюме, где желаемая заработная плата меньше 1 млн рублей.
*Сделайте выводы по представленной диаграмме: для каких уровней образования наблюдаются наибольшие и наименьшие уровни желаемой заработной платы? Как вы считаете, важен ли признак уровня образования при прогнозировании заработной платы?*

In [ ]:
mask = hh_data_w['ЗП (руб)'] < 1000000
data_graf = hh_data_w[mask].groupby('Образование', as_index=False)['ЗП (руб)'].median()
fig = px.bar(
    data_frame=data_graf, #DataFrame
    x = 'Образование',
    y = 'ЗП (руб)', 
    height=500, #высота
    width=1000, #ширина
    color= 'Образование',
    title='Зависимость медианной желаемой заработной платы от уровня образования' #заголовок
)
fig.show()




Наибольший уровень желаемой заработной платы у соскателей с высшим и неоконченным высшим образованием.
Наименьший уровень желаемой заработной платы у соскателей со средним и средне специальным образованием.
Признак уровня образования важен при прогнозировании заработной платы.

5. Постройте диаграмму, которая показывает распределение желаемой заработной платы (**"ЗП (руб)"**) в зависимости от города (**"Город"**). Используйте для диаграммы данные о резюме, где желая заработная плата меньше 1 млн рублей.
*Сделайте выводы по полученной диаграмме: как соотносятся медианные уровни желаемой заработной платы и их размах в городах? Как вы считаете, важен ли признак города при прогнозировании заработной платы?*

In [ ]:
mask = hh_data_w['ЗП (руб)'] < 1000000
data_graf = hh_data_w[mask] 
fig = px.box(
    data_frame=data_graf, #DataFrame
    x = 'ЗП (руб)',
    y = 'Город', 
    height=500, #высота
    width=1000, #ширина
    color= 'Город',
    title='Распределение ЗП' #заголовок
)
fig.show()

На общем фоне медианные уровни желаемой заработной плат выделяются у Москвы и Санкт-Петербурга. В остальном этот признак одинаков.
Признак города при прогнозировании заработной платы важен, если это Москва или Санкт-Петербург).


6. Постройте **многоуровневую столбчатую диаграмму**, которая показывает зависимость медианной заработной платы (**"ЗП (руб)"**) от признаков **"Готовность к переезду"** и **"Готовность к командировкам"**. Проанализируйте график, сравнив уровень заработной платы в категориях.

In [ ]:
data_graf = hh_data_w.groupby(['Готовность к переезду', 'Готовность к командировкам'], as_index=False)['ЗП (руб)'].median()
fig = px.bar(
    data_frame=data_graf, #DataFrame
    y='Готовность к переезду',
    x='ЗП (руб)',
    barmode="group",
    color='Готовность к командировкам',
    orientation='h',
    title='Медианная з/п по готовности к командировкам/переезду'
)
fig.show()
display(hh_data_w.groupby(['Готовность к переезду', 'Готовность к командировкам'], as_index=False)['ЗП (руб)'].median())

Если соискатель готов к перезду и командировкам, то ожидаемая з/п выше

7. Постройте сводную таблицу, иллюстрирующую зависимость **медианной** желаемой заработной платы от возраста (**"Возраст"**) и образования (**"Образование"**). На полученной сводной таблице постройте **тепловую карту**. Проанализируйте тепловую карту, сравнив показатели внутри групп.

In [ ]:
data_graf = hh_data_w.pivot_table(
    values='ЗП (руб)',
    index='Возраст',
    columns='Образование',
    aggfunc='median',
    fill_value=0
).round()
heatmap = sns.heatmap(data=data_graf, cmap='YlGnBu')
heatmap.set_title('Зависимость медианной желаемой заработной платы от возраста и образования', fontsize=16)


При высшем образовании желаемая з/п выше почти по всем возрастам

8. Постройте **диаграмму рассеяния**, показывающую зависимость опыта работы (**"Опыт работы (месяц)"**) от возраста (**"Возраст"**). Опыт работы переведите из месяцев в года, чтобы признаки были в едином масштабе. Постройте на графике дополнительно прямую, проходящую через точки (0, 0) и (100, 100). Данная прямая соответствует значениям, когда опыт работы равен возрасту человека. Точки, лежащие на этой прямой и выше нее - аномалии в наших данных (опыт работы больше либо равен возрасту соискателя)

In [ ]:
fig = px.scatter(
    data_frame=hh_data_w, #DataFrame
    y=hh_data_w['Опыт работы (месяц)'].apply(lambda x: x/12),
    x='Возраст',
    labels = ['Опыт работы', 'Возраст'], 
    title='Зависимость опыта работы от возраста',
)
fig.add_scatter(
    x=[0, 100],
    y=[0, 100],
    mode='lines'
)
fig.show()

С возрастом увеличивается опыт работы

**Дополнительные баллы**

Для получения 2 дополнительных баллов по разведывательному анализу постройте еще два любых содержательных графика или диаграммы, которые помогут проиллюстрировать влияние признаков/взаимосвязь между признаками/распределения признаков. Приведите выводы по ним. Желательно, чтобы в анализе участвовали признаки, которые мы создавали ранее в разделе "Преобразование данных".


In [ ]:
mask = hh_data_w['ЗП (руб)'] < 1000000
data_graf = hh_data_w[mask].groupby('Пол', as_index=False)['ЗП (руб)'].median()
fig = px.bar(
    data_frame=data_graf, #DataFrame
    x = 'Пол',
    y = 'ЗП (руб)', 
    height=500, #высота
    width=1000, #ширина
    color= 'Пол',
    title='Зависимость медианной желаемой заработной платы от пола' #заголовок
)
fig.show()


data_graf = hh_data_w.groupby('Город', as_index=False)['волонтерство'].mean()
fig = px.bar(
    data_frame=data_graf, #DataFrame
    x = 'Город',
    y = 'волонтерство', 
    height=500, #высота
    width=1000, #ширина
    color= 'Город',
    title='Зависимость желания волонтерить от города' #заголовок
)
fig.show()

У мужщин планка з/п выше). В Москве самые не сознительные). Питер - молодцы)


# Очистка данных

1. Начнем с дубликатов в наших данных. Найдите **полные дубликаты** в таблице с резюме и удалите их. 

In [ ]:
dupl_columns = list(hh_data_w.columns)


mask = hh_data_w.duplicated(subset=dupl_columns)
hh_duplicates = hh_data_w[mask]
print(f'Число найденных дубликатов: {hh_duplicates.shape[0]}')

hh_data_w = hh_data_w.drop_duplicates(subset=dupl_columns)

2. Займемся пропусками. Выведите информацию **о числе пропусков** в столбцах. 

In [ ]:
display(hh_data_w.isnull().sum())
display(hh_data_w['Опыт работы (месяц)'].isnull().sum())

3. Итак, у нас есть пропуски в 3ех столбцах: **"Опыт работы (месяц)"**, **"Последнее/нынешнее место работы"**, **"Последняя/нынешняя должность"**. Поступим следующим образом: удалите строки, где есть пропуск в столбцах с местом работы и должностью. Пропуски в столбце с опытом работы заполните **медианным** значением.

In [ ]:
hh_data_w = hh_data_w.dropna(subset=['Последнее/нынешнее место работы', 'Последняя/нынешняя должность'])
hh_data_w = hh_data_w.fillna(value= {'Опыт работы (месяц)': hh_data_w['Опыт работы (месяц)'].median()} )
display(round(hh_data_w['Опыт работы (месяц)'].mean()))

4. Мы добрались до ликвидации выбросов. Сначала очистим данные вручную. Удалите резюме, в которых указана заработная плата либо выше 1 млн. рублей, либо ниже 1 тыс. рублей.

In [ ]:
mask1 = hh_data_w['ЗП (руб)'] > 1000000
mask2 = hh_data_w['ЗП (руб)'] < 1000
display(hh_data_w[mask1 | mask2].shape[0])
mask3 = hh_data_w['ЗП (руб)'] < 1000000
mask4 = hh_data_w['ЗП (руб)'] > 1000
hh_data_w = hh_data_w[mask3 & mask4]



In [ ]:
hh_data_w.info()

5. В процессе разведывательного анализа мы обнаружили резюме, в которых **опыт работы в годах превышал возраст соискателя**. Найдите такие резюме и удалите их из данных


In [ ]:
mask1 = hh_data_w['Опыт работы (месяц)'].apply(lambda x: x/12) > hh_data_w['Возраст']
display(hh_data_w[mask1].shape[0])
mask2 = hh_data_w['Опыт работы (месяц)'].apply(lambda x: x/12) < hh_data_w['Возраст']
hh_data_w = hh_data_w[mask2]

6. В результате анализа мы обнаружили потенциальные выбросы в признаке **"Возраст"**. Это оказались резюме людей чересчур преклонного возраста для поиска работы. Попробуйте построить распределение признака в **логарифмическом масштабе**. Добавьте к графику линии, отображающие **среднее и границы интервала метода трех сигм**. Напомним, сделать это можно с помощью метода axvline. Например, для построение линии среднего будет иметь вид:

`histplot.axvline(log_age.mean(), color='k', lw=2)`

В какую сторону асимметрично логарифмическое распределение? Напишите об этом в комментарии к графику.
Найдите выбросы с помощью метода z-отклонения и удалите их из данных, используйте логарифмический масштаб. Давайте сделаем послабление на **1 сигму** (возьмите 4 сигмы) в **правую сторону**.

Выведите таблицу с полученными выбросами и оцените, с каким возрастом соискатели попадают под категорию выбросов?

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
log_hh = np.log(hh_data_w['Возраст']+1)
histplot = sns.histplot(log_hh, bins=30, ax=ax)
histplot.axvline(log_hh.mean(), color='k', lw=2)
age_up = log_hh.mean()+ 4 * log_hh.std()
age_down = log_hh.mean()- 3 * log_hh.std()
histplot.axvline(age_up, color='k', ls='--', lw=2)
histplot.axvline(age_down, color='k', ls='--', lw=2)
histplot.set_title('Отклонения по возрасту');
display(f'Число выбросов по методу z-отклонения: {hh_data_w[(log_hh < age_down) | (log_hh > age_up)].shape[0]}' )
display(hh_data_w[(log_hh < age_down) | (log_hh > age_up)])
hh_data_w = hh_data_w[(log_hh > age_down) & (log_hh < age_up)]

"Выбросы" - дети от 10 до 15 лет. Ассиметрия в левую сторону.